In [ ]:
#require "yaml"

In [1]:
let yf = 
"key1: val1
key2: val2"

val yf : string = "key1: val1\nkey2: val2"


In [3]:
let y = Yaml.of_string yf

val y : Yaml.value Yaml.res =
  Result.Ok (`O [("key1", `String "val1"); ("key2", `String "val2")])


In [ ]:
let (alpaca_secret, alpaca_key) = 
  let yaml = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") in
  let key = yaml |> get_value "key" |> get_string_opt |> get_string in 
  let secret = yaml |> get_value "secret" |> get_string_opt |> get_string in
  (secret, key)

In [ ]:
type _ gyaml = 
  | Gstring : `String s -> gyaml

File "[2]", line 1, characters 13-14Parse error: [type_kind] expected after "=" (in [opt_eq_pluseq_ctyp])
Uncaught exception:
  
  Exit

Raised at file "camlp4/Camlp4Top/Top.ml", line 61, characters 14-18
Called from file "jupyter/src/repl/evaluation.ml", line 109, characters 24-56
Re-raised at file "parsing/location.ml", line 925, characters 22-25
Called from file "error.cppo.ml", line 147, characters 2-29
Called from file "jupyter/src/repl/evaluation.ml", line 115, characters 14-62
Called from file "jupyter/src/repl/process.ml", line 73, characters 8-80
Called from file "jupyter/src/repl/process.ml", line 76, characters 8-38
Called from file "jupyter/bin/jupyter_main.ml", line 47, characters 4-149


Raised at file "stdlib.ml", line 30, characters 25-45
Called from file "toplevel/toploop.ml", line 493, characters 4-128
Called from file "toplevel/topdirs.ml", line 149, characters 11-23


In [ ]:
let to_gyaml =
  | `String s -> s